### Calculate the luminosity during each segment

$L_{\gamma} = 4\pi D_L ^2 \int_{E_{min}}^{E_{max}} E \dfrac{dN}{dE} dE$

$D_L = 134.1$ $Mpc$

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.table import Table
from astropy import units as u
from scipy import interpolate
from scipy.integrate import quad

In [2]:
emin = 0.1 * u.GeV
emax = 300 * u.GeV
D_l = 134.1 * u.Mpc
D_l = D_l.to(u.cm)
# emin = emin.to(u.erg)
# emax = emax.to(u.erg)

In [3]:
def e_dnde(e, n0, gamma, e0=1):
    return e * n0 * ((e / e0) ** (-gamma))    # Simple power law
    

dirs = ["1-20150918-20160527",  "2-20160913-20170615",  "3A-20170919-20180123",  "3B-20180206-20180622",  "4-20180918-20190528",  "5-20190928-20200608",  "6-20200915-20210503",  "7-20211101-20220619",  "all-20150918-20220619"]


In [4]:
dir = '3B-20180206-20180622/'

# lc_file_path = 'Output/Light_curve_001/4fgl_j0854.8+2006_lightcurve.fits'
lc_file_path = 'Output/Results.fits'
hdul = fits.open('./' + dir+lc_file_path)
lc = hdul[1].data
Table(lc).colnames

['Source_Name',
 'name',
 'class',
 'SpectrumType',
 'SpatialType',
 'SourceType',
 'SpatialModel',
 'RAJ2000',
 'DEJ2000',
 'GLON',
 'GLAT',
 'ts',
 'loglike',
 'npred',
 'offset',
 'offset_ra',
 'offset_dec',
 'offset_glon',
 'offset_glat',
 'offset_roi_edge',
 'pivot_energy',
 'flux_scan',
 'eflux_scan',
 'norm_scan',
 'dloglike_scan',
 'loglike_scan',
 'SpatialWidth',
 'Spatial_Filename',
 'Spectrum_Filename',
 'dec',
 'dec_err',
 'dnde',
 'dnde100',
 'dnde1000',
 'dnde10000',
 'dnde10000_err',
 'dnde10000_index',
 'dnde1000_err',
 'dnde1000_index',
 'dnde100_err',
 'dnde100_index',
 'dnde_err',
 'dnde_index',
 'eflux',
 'eflux100',
 'eflux1000',
 'eflux10000',
 'eflux10000_err',
 'eflux10000_ul95',
 'eflux1000_err',
 'eflux1000_ul95',
 'eflux100_err',
 'eflux100_ul95',
 'eflux_err',
 'eflux_ul95',
 'flux',
 'flux100',
 'flux1000',
 'flux10000',
 'flux10000_err',
 'flux10000_ul95',
 'flux1000_err',
 'flux1000_ul95',
 'flux100_err',
 'flux100_ul95',
 'flux_err',
 'flux_ul95',
 'glat

In [5]:
lc['pivot_energy'][1]

1361.1737551515728

In [6]:
Table(lc)

Source_Name,name,class,SpectrumType,SpatialType,SourceType,SpatialModel,RAJ2000,DEJ2000,GLON,GLAT,ts,loglike,npred,offset,offset_ra,offset_dec,offset_glon,offset_glat,offset_roi_edge,pivot_energy,flux_scan,eflux_scan,norm_scan,dloglike_scan,loglike_scan,SpatialWidth,Spatial_Filename,Spectrum_Filename,dec,dec_err,dnde,dnde100,dnde1000,dnde10000,dnde10000_err,dnde10000_index,dnde1000_err,dnde1000_index,dnde100_err,dnde100_index,dnde_err,dnde_index,eflux,eflux100,eflux1000,eflux10000,eflux10000_err,eflux10000_ul95,eflux1000_err,eflux1000_ul95,eflux100_err,eflux100_ul95,eflux_err,eflux_ul95,flux,flux100,flux1000,flux10000,flux10000_err,flux10000_ul95,flux1000_err,flux1000_ul95,flux100_err,flux100_ul95,flux_err,flux_ul95,glat,glat_err,glon,glon_err,npred_wt,pos_angle,pos_err,pos_err_semimajor,pos_err_semiminor,pos_r68,pos_r95,pos_r99,ra,ra_err,param_names,param_values,param_errors,Flux_Density,Spectral_Index,Pivot_Energy,beta,Exp_Index,Cutoff,Expfactor,Conf_68_PosAng,Conf_68_SemiMajor,Conf_68_SemiMinor,Conf_95_PosAng,Conf_95_SemiMajor,Conf_95_SemiMinor
str48,str48,str32,str32,str32,str32,str32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64[20],float64[20],float64[20],float64[20],float64[20],float64,str128,str128,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str32[10],float64[10],float64[10],float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
4FGL J0854.8+2006,4FGL J0854.8+2006,BLL,LogParabola,SkyDirFunction,PointSource,PointSource,133.7071075439453,20.11590003967285,206.80487060546875,35.82643127441406,154.59654116992897,-44064.36278320202,249.32652116797567,0.00807118322700262,-0.0032509335281855067,0.007389240103319539,0.005877065235041501,0.005529199038411647,-7.4926107598970075,803.7171733140149,0.0 .. 5.1075921853685046e-08,0.0 .. 2.449743541960998e-05,0.0 .. 0.9110534204978386,-80.35229035567318 .. -2.8292108364839805,-44144.711392270736 .. -44067.18831275155,nan,,,20.11590003967285,nan,5.316136276342862e-12,4.233341103164801e-10,3.3114108719025156e-12,1.911561862128855e-14,5.62612756778097e-15,2.304601973109555,3.409819393855806e-13,2.1726474530390236,1.2103190547328416e-10,2.0406929330348214,5.399671355207201e-13,2.1726474530390236,1.862063676138851e-05,1.864595864733929e-05,9.781309109328215e-06,3.764873345865785e-06,2.356800399286963e-06,4.573110332470812e-06,3.01525084519851e-06,1.1881144900154194e-05,3.3652561792729036e-06,2.2648843218751564e-05,3.3353160593558745e-06,2.2618085270836442e-05,3.882309196044308e-08,3.882349124603449e-08,2.7186181905517297e-09,1.4107123372813948e-10,6.278721754076188e-11,1.7135618049010413e-10,3.539318558498825e-10,3.302246794279924e-09,6.685923657711601e-09,4.715805623442606e-08,6.685893263768183e-09,4.7157571230842687e-08,35.82643127441406,nan,206.80487060546875,nan,249.32652116797567,nan,nan,nan,nan,nan,nan,nan,133.7071075439453,nan,norm ..,6.924967664056352e-12 .. nan,7.066898404780053e-13 .. nan,6.924967664056352e-12,2.153099919604749,710.9868774414062,0.0286535599550497,nan,nan,nan,nan,nan,nan,nan,nan,nan
4FGL J0856.8+2056,4FGL J0856.8+2056,bll,PowerLaw,SkyDirFunction,PointSource,PointSource,134.20030212402344,20.94610023498535,206.02212524414062,36.53593444824219,11.101515950082103,-44064.36297261661,97.71096713949515,0.9580668210983276,-0.46384756505917,0.8382748629726371,0.634761574232777,0.7175856989497214,-6.661725137027682,1361.1737551515728,0.0 .. 3.26000660828428e-08,0.0 .. 8.954980890623548e-06,0.0 .. 1.3416225997183706,-6.012628375778149 .. -2.74098753787257,-44070.37173029084 .. -44067.10

In [7]:
params = pd.read_csv('param.csv', index_col=0, sep='&', header=0)

In [8]:
params

,TS,Flux,Flux_err,eFlux,eFlux_err,log_N0,log_N0_err,g,g_err
Seg,,,,,,,,,
1,1586.61,10.79,0.70,0.46,0.03,-9.51,0.02,2.14,0.04
2,891.89,4.07,0.05,0.35,0.01,-9.94,0.05,1.92,0.05
3A,155.29,3.44,0.95,0.15,0.03,-9.89,0.16,2.19,0.19
3B,154.60,3.88,0.67,0.19,0.03,-10.06,0.12,1.97,0.12
4,263.04,4.80,0.78,0.20,0.03,-9.91,0.06,2.21,0.10
5,379.65,4.52,0.72,0.26,0.05,-9.96,0.05,2.10,0.07
6,308.48,3.72,0.67,0.21,0.03,-9.97,0.06,2.03,0.08
7,320.09,2.06,0.25,0.19,0.03,-10.20,0.13,1.84,0.12


In [9]:
luminosity = []
for i in range(0, len(dirs)-1):
    idx = dirs[i].split('-')[0]
    print(idx)
    lc_file_path = 'Output/Results.fits'
    hdul = fits.open('./' + dirs[i]+'/'+lc_file_path)
    lc = hdul[1].data
    e0 = lc['pivot_energy'][1] * u.MeV
    n0 = 10 ** (params['log_N0'][idx]) * (1/(u.cm**2 * u.s * u.MeV))
    gamma = params['g'][idx]
    # print(n0, gamma, e0)
    res = 4*np.pi * (D_l**2) * quad(e_dnde, emin.value, emax.value, args=(n0.to(1/(u.cm**2 * u.s * u.GeV)).value, gamma, e0.to(u.GeV).value), epsabs=1e-18, epsrel=1e-18, limit=100000) * u.GeV/(u.cm**2 * u.s)
    print([res.to(u.erg/u.s).value[0], res.value[0]])
    # print(res)
    luminosity.append([res.to(u.erg/u.s).value[0], res.value[0]])


1
[1.854771137408575e+46, 1.1576570885183533e+49]
2
[2.197304804022656e+46, 1.3714497873663638e+49]
3A
[1.0245231277187164e+45, 6.394570398651292e+47]
3B
[4.655050953583304e+45, 2.9054542768867422e+48]
4
[8.74875913332741e+45, 5.460545952093451e+48]
5
[2.6735920705446492e+45, 1.668724916971077e+48]
6
[1.409204319896751e+46, 8.79556155040488e+48]
7
[1.2141981878111745e+46, 7.578429007417261e+48]


In [10]:
final_table = pd.read_csv('final_table.tex', index_col=0, sep = ' ', header=None)

In [11]:
final_table

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
0,,,,,,,,,,,,,,,,,,,,,
1,&,1586.61,&,10.79,$\pm$,0.70,&,0.46,$\pm$,0.03,...,$\pm$,0.02,&,2.14,$\pm$,0.04,&,18.55,11.58,\\
2,&,891.89,&,4.07,$\pm$,0.05,&,0.35,$\pm$,0.01,...,$\pm$,0.05,&,1.92,$\pm$,0.05,&,21.97,13.71,\\
3,&,155.29,&,3.44,$\pm$,0.95,&,0.15,$\pm$,0.03,...,$\pm$,0.16,&,2.19,$\pm$,0.19,&,1.02,0.64,\\
3,&,154.60,&,3.88,$\pm$,0.67,&,0.19,$\pm$,0.03,...,$\pm$,0.12,&,1.97,$\pm$,0.12,&,4.66,2.91,\\
4,&,263.04,&,4.80,$\pm$,0.78,&,0.20,$\pm$,0.03,...,$\pm$,0.06,&,2.21,$\pm$,0.10,&,8.75,5.46,\\
5,&,379.65,&,4.52,$\pm$,0.72,&,0.26,$\pm$,0.05,...,$\pm$,0.05,&,2.10,$\pm$,0.07,&,2.67,1.67,\\
6,&,308.48,&,3.72,$\pm$,0.67,&,0.21,$\pm$,0.03,...,$\pm$,0.06,&,2.03,$\pm$,0.08,&,14.09,8.80,\\
7,&,320.09,&,2.06,$\pm$,0.25,&,0.19,$\pm$,0.03,...,$\pm$,0.13,&,1.84,$\pm$,0.12,&,12.14,7.58,\\


In [12]:
final_table[19] = [' & '] * len(final_table)
final_table[20] = [np.round(luminosity[i][0]/1e45, 2) for i in range(0, len(dirs)-1)]
final_table[21] = [' & '] * len(final_table)
final_table[22] = [np.round(luminosity[i][1]/1e48, 2) for i in range(0, len(dirs)-1)]
final_table[23] = [' \\\\'] * len(final_table)

In [13]:
final_table.to_csv('final_table.tex', sep=' ', header=None)